In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from transformers import XLNetTokenizer, XLNetForSequenceClassification, get_scheduler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
file_path = "/content/drive/My Drive/emotion_dataset.csv"  # Adjust your file path
df = pd.read_csv(file_path)

# Display first few rows
print(df.head())

Using device: cuda
   Unnamed: 0                                               text Emotion
0           0  i seriously hate one subject to death but now ...    hate
1           3  ive been really angry with him and i feel like...   anger
2           5  i feel jealous becasue i wanted that kind of l...    love
3           7  i finally fell asleep feeling angry useless an...   worry
4          15               i feel like they hated me since then    hate


In [ ]:
# Encode emotion labels
emotion_labels = df["Emotion"].unique().tolist()
label_to_id = {label: idx for idx, label in enumerate(emotion_labels)}
df["label"] = df["Emotion"].map(label_to_id)

# Split dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"].tolist(), df["label"].tolist(), test_size=0.2, random_state=42
)

# Load XLNet tokenizer
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

# Define dataset class
class EmotionDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, padding="max_length", truncation=True, max_length=self.max_len, return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Create datasets and dataloaders
train_dataset = EmotionDataset(train_texts, train_labels, tokenizer)
test_dataset = EmotionDataset(test_texts, test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# Load XLNet model
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=len(emotion_labels))
model.to(device)

# Define optimizer, loss function, and scheduler
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()

num_training_steps = len(train_loader) * 3  # Assuming 3 epochs
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

scaler = GradScaler()


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-71a49ccd92f9>:55: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScale

In [ ]:
# Training function with AMP
def train(model, train_loader, optimizer, loss_fn, epochs=1):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            with autocast():
                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss  # Directly get loss from model

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            lr_scheduler.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

# Train model
train(model, train_loader, optimizer, loss_fn)


<ipython-input-7-871e4104f4a1>:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1, Loss: 0.0961


In [ ]:
def evaluate(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    total_correct = 0
    total_examples = 0

    with torch.no_grad():  # No need to compute gradients for evaluation
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            # Get model outputs
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Choose the class with the highest logit as prediction
            predictions = torch.argmax(logits, dim=1)

            # Update the count of correct predictions and total examples
            total_correct += (predictions == labels).sum().item()
            total_examples += labels.size(0)

    # Calculate accuracy
    accuracy = total_correct / total_examples
    return accuracy

# Evaluate the model on the test set
test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.4f}")


Test Accuracy: 0.9976


In [ ]:
# Save the model to Google Drive
model_save_path = "/content/drive/My Drive/emotional_analysis.pt"  # Path on your Google Drive
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to: {model_save_path}")